## Setup

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf

## TensorFlow Core Walkthrough

### Graph

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


### TensorBoard

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

### Session

In [4]:
sess = tf.Session()
print(sess.run(total))

7.0


In [5]:
print(sess.run({'ab': (a, b), 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [6]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.56197965 0.7048255  0.7499045 ]
[0.83280253 0.72722375 0.7294365 ]
(array([1.1648749, 1.8248862, 1.7813193], dtype=float32), array([2.164875 , 2.8248863, 2.7813191], dtype=float32))


### Feeding

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [8]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y:[2, 4]}))

7.5
[3. 7.]


## Datasets

In [9]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [10]:
r = tf.random_normal([10, 3]) # need to be initalized
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()
sess.run(iterator.initializer)

while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

[ 1.5301434  -1.2222139  -0.18496378]
[-0.34257147  0.7699402  -2.559804  ]
[-0.37470984 -0.47780067 -0.15916249]
[0.7415607  0.90958315 0.03753797]
[-2.185148   -0.40521502 -1.5375686 ]
[-0.2719819   0.73091865  0.4428518 ]
[-1.0287449 -0.7210024  0.4634773]
[ 0.70746243  0.15897769 -0.6963033 ]
[ 0.82683885 -0.28732592  0.06796919]
[ 0.67661273 -0.2025682  -0.17871903]


## Layers

### Creating Layers

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

### Initializing Layers

In [12]:
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[ -5.192934 ]
 [-12.0343895]]


In [14]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y, {x: [[1, 2, 3,], [4, 5, 6]]}))

[[2.2378435]
 [4.2076697]]


## Feature columns

In [15]:
features = {
    'sales': [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']
}
department_column = tf.feature_column.categorical_column_with_vocabulary_list('department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)
columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]
inputs = tf.feature_column.input_layer(features, columns)

In [16]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [17]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


## Training

### Define the data

In [18]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the model

In [19]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

In [20]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y_pred))

[[-0.42881382]
 [-0.85762763]
 [-1.2864414 ]
 [-1.7152553 ]]


### Loss

In [21]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

0.5909715


In [22]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [23]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

0.5909715
0.46750093
0.38148385
0.32145715
0.2794664
0.2499927
0.22920626
0.21444967
0.20387915
0.19621521
0.19057001
0.18632755
0.18306041
0.18047184
0.17835617
0.17657049
0.17501569
0.17362294
0.17234458
0.17114742
0.17000845
0.16891168
0.16784611
0.16680393
0.1657798
0.16477
0.16377197
0.16278383
0.16180438
0.16083264
0.15986806
0.15891016
0.15795863
0.15701322
0.1560738
0.15514015
0.15421224
0.15329
0.15237334
0.15146223
0.15055656
0.14965639
0.14876156
0.14787209
0.14698797
0.14610913
0.1452356
0.14436723
0.14350408
0.14264607
0.14179322
0.14094546
0.1401028
0.13926512
0.13843247
0.1376048
0.1367821
0.1359643
0.13515137
0.13434333
0.13354012
0.13274166
0.13194805
0.13115916
0.13037495
0.1295955
0.12882066
0.12805045
0.12728488
0.12652384
0.12576737
0.12501544
0.124268025
0.123525016
0.1227865
0.12205239
0.121322624
0.12059726
0.119876236
0.1191595
0.118447065
0.11773892
0.11703495
0.11633523
0.11563967
0.114948265
0.114261016
0.1135779
0.1128988
0.11222381
0.11155283
0.110885866
0

### Complete program

In [24]:
# x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
# y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)
# linear_model = tf.layers.Dense(units=1)
# y_pred = linear_model(x)
# loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
# optimizer = tf.train.GradientDescentOptimizer(0.01)
# train = optimizer.minimize(loss)
# init = tf.global_variables_initializer()
# sess = tf.Session()
# sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)
print(sess.run(y_pred))

0.10505988
0.10443175
0.10380733
0.10318669
0.102569774
0.10195654
0.10134693
0.100741
0.100138694
0.09953998
0.09894485
0.09835327
0.09776526
0.097180694
0.09659968
0.09602213
0.09544804
0.09487736
0.09431012
0.09374624
0.093185745
0.09262861
0.092074804
0.09152427
0.09097706
0.09043313
0.089892454
0.08935498
0.08882077
0.08828969
0.08776183
0.087237135
0.08671555
0.08619708
0.08568172
0.08516945
0.08466025
0.08415409
0.08365092
0.083150804
0.08265364
0.082159474
0.08166824
0.081179984
0.08069459
0.080212146
0.07973258
0.07925585
0.07878201
0.07831098
0.07784278
0.077377364
0.07691472
0.07645486
0.07599775
0.075543374
0.07509171
0.07464274
0.07419649
0.07375288
0.07331192
0.072873585
0.07243788
0.072004795
0.07157431
0.07114635
0.07072098
0.07029818
0.06987787
0.06946007
0.06904477
0.06863196
0.06822162
0.06781375
0.06740829
0.067005254
0.06660464
0.066206425
0.06581061
0.06541713
0.065026015
0.06463724
0.06425078
0.06386663
0.0634848
0.06310522
0.06272793
0.062352877
0.061980076
0.06